### Connecting to postgresql

In [1]:
## Connecting to postgresql
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

## set connection
#%reload_ext sql
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

 * Starting PostgreSQL 14 database server
   ...done.
CREATE ROLE


### Create and insert data to table

In [4]:
# Create database
!sudo -u postgres psql -c "CREATE DATABASE superstore_db"

CREATE DATABASE


In [5]:
# Check if sampledb has been added to our collection of databases
%sql SELECT datname FROM pg_database

 * postgresql+psycopg2://@/postgres


,datname
0,postgres
1,superstore_db
2,template1
3,template0


In [6]:
# Connect to the database
!sudo -u postgres psql -c "\c superstore_db"

You are now connected to database "superstore_db" as user "postgres".


In [7]:
# Drop table 'sales' if already existing
%%sql
DROP TABLE IF EXISTS sales;

 * postgresql+psycopg2://@/postgres


""


In [9]:
%%sql
create table sales
(
  RowID             int,
  OrderID          varchar,
  OrderDate        date,
  ShipDate         date,
  ShipMode         varchar,
  CustomerID       varchar,
  CustomerName     varchar,
  Segment           varchar,
  Country           varchar,
  City              varchar,
  State             varchar,
  PostalCode        int,
  Region            varchar,
  ProductID        varchar,
  Category          varchar,
  SubCategory      varchar,
  ProductName      varchar,
  Sales            float,
  Quantity           int,
  Discount         float,
  Profit           float
);

 * postgresql+psycopg2://@/postgres


""


In [10]:
# Loading data feom csv file
%%sql
copy sales
from '/content/sales2.csv'
delimiter ','
csv header;

 * postgresql+psycopg2://@/postgres


""


### Display first 5 rows

In [11]:
%%sql
select *
from sales
limit 5;

 * postgresql+psycopg2://@/postgres


,rowid,orderid,orderdate,shipdate,shipmode,customerid,customername,segment,country,city,...,postalcode,region,productid,category,subcategory,productname,sales,quantity,discount,profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


### Question 1: Which are the top 10 selling products?

In [ ]:
%%sql
select productname, count(orderid) as num_sales
from sales
group by productname
order by num_sales desc
limit 10;

 * postgresql+psycopg2://@/postgres


,productname,num_sales
0,Staple envelope,48
1,Staples,46
2,Easy-staple paper,46
3,Avery Non-Stick Binders,20
4,Staples in misc. colors,19
5,KI Adjustable-Height Table,18
6,Staple remover,18
7,Storex Dura Pro Binders,17
8,Staple-based wall hangings,16
9,Logitech 910-002974 M325 Wireless Mouse for We...,15


### Question 2: Which are the top 10 profitable products?

In [ ]:
%%sql
select productname, sum(profit) as total_profit
from sales
group by productname
order by total_profit desc
limit 10;

 * postgresql+psycopg2://@/postgres


,productname,total_profit
0,Canon imageCLASS 2200 Advanced Copier,25199.9280
1,Fellowes PB500 Electric Punch Plastic Comb Bin...,7753.0390
2,Hewlett Packard LaserJet 3310 Copier,6983.8836
3,Canon PC1060 Personal Laser Copier,4570.9347
4,HP Designjet T520 Inkjet Large Format Printer ...,4094.9766
5,Ativa V4110MDD Micro-Cut Shredder,3772.9461
6,"3D Systems Cube Printer, 2nd Generation, Magenta",3717.9714
7,Plantronics Savi W720 Multi-Device Wireless He...,3696.2820
8,Ibico EPK-21 Electric Binding System,3345.2823
9,Zebra ZM400 Thermal Label Printer,3343.5360


### Question 3: Which category of product was sold the most?


In [ ]:
%%sql
select category, count(orderid) as num_sales
from Sales
group by category
order by num_sales desc;

 * postgresql+psycopg2://@/postgres


,category,num_sales
0,Office Supplies,6026
1,Furniture,2121
2,Technology,1847


### Question 4: Most profitable category?

In [ ]:
%%sql
select category, sum(profit) as total_profit
from Sales
group by category
order by total_profit desc;

 * postgresql+psycopg2://@/postgres


,category,total_profit
0,Technology,145454.9481
1,Office Supplies,122490.8008
2,Furniture,18451.2728


### Questin 5: Evaluate the total sales value by category and sub category.

In [ ]:
%%sql
select category, subcategory, sum(sales) as total_sales_value
from sales
group by category, subcategory
order by category, total_sales_value desc;

 * postgresql+psycopg2://@/postgres


,category,subcategory,total_sales_value
0,Furniture,Chairs,328449.1030
1,Furniture,Tables,206965.5320
2,Furniture,Bookcases,114879.9963
3,Furniture,Furnishings,91705.1640
4,Office Supplies,Storage,223843.6080
5,Office Supplies,Binders,203412.7330
6,Office Supplies,Appliances,107532.1610
7,Office Supplies,Paper,78479.2060
8,Office Supplies,Supplies,46673.5380
9,Office Supplies,Art,27118.7920


### Question 6: Most selling product in each subcategory?

In [ ]:
%%sql
with rank_table as
(
select subcategory,
       productname,
       count(orderid) as num_sales,
       rank() over(partition by subcategory order by count(orderid) desc) rank
from sales
group by subcategory, productname
order by subcategory, rank
)
select subcategory, productname, num_sales, rank
from rank_table
where rank=1
order by num_sales desc
;

 * postgresql+psycopg2://@/postgres


,subcategory,productname,num_sales,rank
0,Envelopes,Staple envelope,48,1
1,Paper,Easy-staple paper,46,1
2,Fasteners,Staples,46,1
3,Binders,Avery Non-Stick Binders,20,1
4,Art,Staples in misc. colors,19,1
5,Tables,KI Adjustable-Height Table,18,1
6,Supplies,Staple remover,18,1
7,Furnishings,Staple-based wall hangings,16,1
8,Accessories,Logitech 910-002974 M325 Wireless Mouse for We...,15,1
9,Chairs,"Situations Contoured Folding Chairs, 4/Set",15,1


### Question 7: Which customer segment is most profitable?

In [ ]:
%%sql
select segment, sum(profit) as total_profit
from sales
group by segment
order by total_profit desc;

 * postgresql+psycopg2://@/postgres


,segment,total_profit
0,Consumer,134119.2092
1,Corporate,91979.1340
2,Home Office,60298.6785


### Question 8: Which shipping mdes sold most products?


In [ ]:
%%sql
select shipmode, count(orderid) as num_sales
from sales
group by shipmode
order by num_sales desc;

 * postgresql+psycopg2://@/postgres


,shipmode,num_sales
0,Standard Class,5968
1,Second Class,1945
2,First Class,1538
3,Same Day,543


### Question 9: What market sold the most products

In [ ]:
%%sql
select region, count(orderid) as num_sales
from sales
group by region
order by num_sales desc;

 * postgresql+psycopg2://@/postgres


,region,num_sales
0,West,3203
1,East,2848
2,Central,2323
3,South,1620


### Question 10: Top 10 cities by sales

In [ ]:
%%sql
select city, sum(sales) as total_sales
from sales
group by city
order by total_sales desc
limit 10;

 * postgresql+psycopg2://@/postgres


,city,total_sales
0,New York City,256368.1610
1,Los Angeles,175851.3410
2,Seattle,119540.7420
3,San Francisco,112669.0920
4,Philadelphia,109077.0130
5,Houston,64504.7604
6,Chicago,48539.5410
7,San Diego,47521.0290
8,Jacksonville,44713.1830
9,Springfield,43054.3420


### Question 11: Total sales value by year and month

In [ ]:
%%sql
select date_part('year', orderdate) as year,
       date_part('month', orderdate) as month,
       sum(sales) as total_sales_value
from sales
group by year, month
order by year, month;


 * postgresql+psycopg2://@/postgres


,year,month,total_sales_value
0,2014.0,1.0,14236.8950
1,2014.0,2.0,4519.8920
2,2014.0,3.0,55691.0090
3,2014.0,4.0,28295.3450
4,2014.0,5.0,23648.2870
5,2014.0,6.0,34595.1276
6,2014.0,7.0,33946.3930
7,2014.0,8.0,27909.4685
8,2014.0,9.0,81777.3508
9,2014.0,10.0,31453.3930


### Question 12 : Find top 2 months in which we have high sales in each year?

In [ ]:
%%sql
with temp_table as
(
select date_part('year', orderdate) as year,
       date_part('month', orderdate) as month,
       sum(sales) as total_sales_value,
       rank() over(partition by date_part('year', orderdate) order by sum(sales) desc) rank
from sales
group by year, month
)
select year, month, total_sales_value, rank
from temp_table
where rank<=2;

 * postgresql+psycopg2://@/postgres


,year,month,total_sales_value,rank
0,2014.0,9.0,81777.3508,1
1,2014.0,11.0,78628.7167,2
2,2015.0,11.0,75972.5635,1
3,2015.0,12.0,74919.5212,2
4,2016.0,12.0,96999.0430,1
5,2016.0,11.0,79411.9658,2
6,2017.0,11.0,118447.8250,1
7,2017.0,9.0,87866.6520,2
